<h1>Extract

<h4>Loading the file and extracting the columns that we are interested in: "rating" and "variation".

In [1]:
import pandas as pd

csv_file_path = "data/amazon_alexa.csv"

data = pd.read_csv(csv_file_path)

ratings_df = data[["rating", "variation"]]

ratings_df.head()

,rating,variation
0,5,Charcoal Fabric
1,5,Charcoal Fabric
2,4,Walnut Finish
3,5,Charcoal Fabric
4,5,Charcoal Fabric


<h1> Transform

<h4> Checking the given label to the Alexa products.

In [130]:
unique_variations = ratings_df['variation'].unique()
print(unique_variations)


['Charcoal Fabric ' 'Walnut Finish ' 'Heather Gray Fabric '
 'Sandstone Fabric ' 'Oak Finish ' 'Black' 'White' 'Black  Spot'
 'White  Spot' 'Black  Show' 'White  Show' 'Black  Plus' 'White  Plus'
 'Configuration: Fire TV Stick' 'Black  Dot' 'White  Dot']


<h4> Replacing the colour variations with the model name in order to normalize our data.

In [134]:
replacement_dict = {
    'Black  Dot': 'Echo Dot',
    'White  Dot': 'Echo Dot',
    'Black  Spot': 'Echo Dot',
    'White  Spot': 'Echo Dot',
    'White  Show': 'Echo Show',
    'Black  Show': 'Echo Show',
    'Heather Gray Fabric ': 'Echo Dot',
    'Sandstone Fabric ': 'Echo',
    'Walnut Finish ': 'Echo',
    'Oak Finish ': 'Echo',
    'Charcoal Fabric ': 'Echo',
    'Configuration: Fire TV Stick': 'Fire TV Stick',
    'White Plus': 'Echo Plus',
    'Black Plus': 'Echo Plus',
    'White': 'Echo',
    'Black': 'Echo',
    'White  Plus': 'Echo Plus',
    'Black  Plus': 'Echo Plus',
    
    
}

ratings_df.loc[:, 'variation'] = ratings_df['variation'].replace(replacement_dict)

unique_variations = ratings_df['variation'].unique()
print(unique_variations)

['Echo' 'Echo Dot' 'Echo Show' 'Echo Plus' 'Fire TV Stick']


<h4>Calculating the average mean for each product and renaming the columns.

In [155]:
average_ratings_df = ratings_df.groupby('variation')['rating'].mean().round(decimals = 2)

average_ratings_df = average_ratings_df.reset_index()

average_ratings_df.columns = ['Product', 'Average_Rating']


print(average_ratings_df)


         Product  Average_Rating
0           Echo            4.49
1       Echo Dot            4.44
2      Echo Plus            4.37
3      Echo Show            4.44
4  Fire TV Stick            4.59


<h4>Creating a new column for couting the reviews for each product.

In [156]:
product_counts = ratings_df['variation'].value_counts().reset_index()
product_counts.columns = ['Product', 'Count']
alexa_rating_df = average_ratings_df.merge(product_counts, on='Product')

print(alexa_rating_df)


         Product  Average_Rating  Count
0           Echo            4.49    895
1       Echo Dot            4.44   1207
2      Echo Plus            4.37    348
3      Echo Show            4.44    350
4  Fire TV Stick            4.59    350


<h1>Load

In [169]:
import sqlite3

<h4>Connecting to our SQL database/creating it in case it doesn't exist already.

In [170]:
conn = sqlite3.connect('amazon_alexa_rating.db')
cursor = conn.cursor()

<h4>Creating a SQL table that matches our pandas dataframe.

In [171]:
create_table_sql = '''
    CREATE TABLE IF NOT EXISTS alexa_product_ratings (
        Product TEXT,
        Average_Rating REAL,
        Count INTEGER
    )
'''


In [172]:
conn.execute(create_table_sql)


<h4>Loading our dataframe to the SQL database.

In [173]:
alexa_rating_df.to_sql('alexa_product_ratings', conn, if_exists='replace', index=False)


5

<h4>Commiting our changes and closing the database connection.

In [ ]:
conn.commit()
conn.close()